In [1]:
import numpy as np
import gurobipy as gp
import pandas as pd

In [3]:
def calcula_distancias(nome):
    with open('Instancias/' + nome, 'r') as f:
        tsp = f.readlines()
    coord = list(map(lambda x: x.strip().split(' ')[1:],tsp[2:]))
    coord = list(map(lambda x: [int(x[i]) for i in range(len(x))],coord))
    dict_distancias = {(u,v): np.linalg.norm([coord[u][0]-coord[v][0],coord[u][1]-coord[v][1]]) for u in range(len(coord)) for v in range(len(coord))}
    n_variaveis = len(coord)
    return dict_distancias,n_variaveis

In [4]:
def cria_modelo(dict_distancias, n_variaveis):
    #Criando Rótulos
    rotulos = list(range(n_variaveis))
    
    #Criando Modelo
    m = gp.Model()

    #Criando variáveis
    x = m.addVars(rotulos,rotulos, vtype = gp.GRB.BINARY)
    u = m.addVars(rotulos[1:])

    # Função Objetivo
    m.setObjective(
        gp.quicksum(dict_distancias[(i,j)]*x[i,j]for j in rotulos for i in rotulos if i!=j ),
        sense = gp.GRB.MINIMIZE
    )

    #Adicionando restrições

    #Conservação de fluxo

    m.addConstrs(
        gp.quicksum(x[i,j] for i in rotulos if j!=i) == 1
        for j in rotulos
    )
    m.addConstrs(
        gp.quicksum(x[i,j] for j in rotulos if j!=i) == 1
        for i in rotulos
    )

    # Eliminação de sub-rotas Miller-Tucler

    m.addConstrs((u[i]-u[j]+n_variaveis*x[i,j]) <= n_variaveis-1
                 for i in rotulos[1:]
                 for j in rotulos[1:] if i!=j)
    m.setParam("TimeLimit", 1200)
    return m, x

In [5]:
def solver(tamanho):
    resultados = []
    nomes = [f'instancia{tamanho}x{tamanho}-{i}.txt' for i in range(1,11)]
    k =  1
    for nome in nomes:
        dict_distancias,n_variaveis = calcula_distancias(nome)
        m,x = cria_modelo(dict_distancias, n_variaveis)
        m.optimize()
        parcial = [m.ObjVal, m.MipGap, m.runtime].copy()
        resultados.append(parcial)
        matriz_adjacente = []
        for i in range(n_variaveis):
            lista_i = []
            for j in range(n_variaveis):
                lista_i.append(round(x[i,j].X))
            matriz_adjacente.append(lista_i)
        df = pd.DataFrame(matriz_adjacente, index = range(n_variaveis), columns = range(n_variaveis))
        df.to_csv(f'matriz_adjacente_{tamanho}x{tamanho}-{k}.csv')
        k+=1
    return resultados

In [25]:
resultado_10x10 = solver(10)

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 92 rows, 109 columns and 396 nonzeros
Model fingerprint: 0x4f27745e
Variable types: 9 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Found heuristic solution: objective 772.1346652
Presolve removed 0 rows and 10 columns
Presolve time: 0.00s
Presolved: 92 rows, 99 columns, 396 nonzeros
Variable types: 9 continuous, 90 integer (90 binary)

Root relaxation: objective 4.714937e+02, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | 

In [27]:
resultado_25x25 = solver(25)

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 602 rows, 649 columns and 2856 nonzeros
Model fingerprint: 0x4c74d470
Variable types: 24 continuous, 625 integer (625 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 0 rows and 25 columns
Presolve time: 0.01s
Presolved: 602 rows, 624 columns, 2856 nonzeros
Variable types: 24 continuous, 600 integer (600 binary)

Root relaxation: objective 6.945728e+02, 73 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  694.57279  

In [28]:
resultado_50x50 = solver(50)

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2452 rows, 2549 columns and 11956 nonzeros
Model fingerprint: 0x75692440
Variable types: 49 continuous, 2500 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [5e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 0 rows and 50 columns
Presolve time: 0.02s
Presolved: 2452 rows, 2499 columns, 11956 nonzeros
Variable types: 49 continuous, 2450 integer (2450 binary)

Root relaxation: objective 8.888556e+02, 165 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  

In [29]:
resultado_10x10

[[558.7679519826171, 0.0, 0.06999993324279785],
 [418.2990688564662, 1.5124090793793368e-05, 0.14999985694885254],
 [552.8997142950096, 0.0, 0.29600000381469727],
 [642.5388031937047, 0.0, 0.06500005722045898],
 [474.7046486081207, 0.0, 0.12100005149841309],
 [575.8188299484352, 0.0, 0.05200004577636719],
 [418.23867332129043, 0.0, 0.11299991607666016],
 [568.2728742000683, 0.0, 0.05200004577636719],
 [537.9529871965179, 0.0, 0.25300002098083496],
 [535.1746643400477, 0.0, 0.0989999771118164]]

In [31]:
resultado_25x25

[[821.8249230330025, 0.0, 1.49399995803833],
 [857.3790042109639, 0.0, 0.9509999752044678],
 [730.1485573405296, 0.0, 0.6919999122619629],
 [879.5265568053171, 0.0, 4.047999858856201],
 [799.5956456290666, 0.0, 3.7309999465942383],
 [916.8969376861521, 0.0, 8.559000015258789],
 [852.32629490775, 0.0, 0.21599984169006348],
 [800.3320860073426, 0.0, 2.4820001125335693],
 [851.0251588418901, 0.0, 3.5429999828338623],
 [866.6355745197066, 0.0, 0.6590001583099365]]

In [32]:
resultado_50x50

[[1141.509655565358, 0.0, 39.02900004386902],
 [1112.9449275038203, 0.0, 203.32599997520447],
 [1175.9446744877218, 0.0, 14.611000061035156],
 [1217.6436333180618, 0.0, 216.93799996376038],
 [1168.669922166123, 0.0, 29.776000022888184],
 [1164.9331026401123, 0.0, 279.07800006866455],
 [1171.4450599381667, 0.0, 18.68399977684021],
 [1170.4508118145648, 0.0, 18.546000003814697],
 [1115.1840559621216, 0.0, 209.04700016975403],
 [1099.7009642185724, 0.0, 7.289999961853027]]

In [6]:
resultado_100x100 = solver(100)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-01
Set parameter TimeLimit to value 1200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9902 rows, 10099 columns and 48906 nonzeros
Model fingerprint: 0xada9cc33
Variable types: 99 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 0 rows and 100 columns
Presolve time: 0.07s
Presolved: 9902 rows, 9999 columns, 48906 nonzeros
Variable types: 99 continuous, 9900 integer (9900 binary)

Root relaxation: objective 1.054087e+03, 353 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wo

In [7]:
resultado_100x100

[[1449.911606790934, 0.0, 115.64900016784668],
 [1628.3221400428374, 0.0, 707.5999999046326],
 [1529.8264238124707, 0.0, 478.1599998474121],
 [1507.353726913816, 0.0243219173337269, 1200.095999956131],
 [1590.8030160414187, 0.019152964600718844, 1200.0929999351501],
 [1572.70621642393, 0.02789644145243506, 1200.1009998321533],
 [1666.1200672626433, 0.0, 582.3560001850128],
 [1449.2418658390693, 0.0, 262.6510000228882],
 [1516.3931279405722, 0.005069528824581041, 1200.09099984169],
 [1584.8545329315227, 0.0, 250.4579999446869]]